In [2]:
import pandas as pd

#training set 

train_data_file_name = "C:\Users\hp\Documents\Projects5\classifiers\Dataset3.csv"
train_data_df = pd.read_csv(train_data_file_name, header=None, delimiter=",")
train_data_df.columns = ["Sentiment","Text"]

#testing set 

test_data_file_name = "C:\Users\hp\Documents\Projects5\Testdata.csv"
test_data_df = pd.read_csv(test_data_file_name, header=None, delimiter="\n")
test_data_df.columns = ["Text"]

In [3]:
train_data_df.shape

(10000, 2)

In [4]:
test_data_df.shape

(787952, 1)

In [5]:
train_data_df.head()


,Sentiment,Text
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,0,.. Omgaga. Im sooo im gunna CRy. I'...
3,0,i think mi bf is cheating on me!!! ...
4,0,or i just worry too much?


In [6]:
test_data_df.head()

,Text
0,omg its already 7:30 :O
1,Juuuuuuuuuuuuuuuuussssst Chillin!!
2,handed in my uniform today . i miss you ...
3,hmmmm.... i wonder how she my number @-)
4,thanks to all the haters up in my face a...


In [7]:
train_data_df.Sentiment.value_counts()

1    5000
0    5000
Name: Sentiment, dtype: int64

In [8]:
import numpy as np 

np.mean([len(s.split(" ")) for s in train_data_df.Text])

13.786099999999999

In [19]:
import re, nltk
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer

#######
# based on http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    # remove non letters
    text = re.sub("[^a-zA-Z]", " ", text)
    # tokenize
    tokens = nltk.word_tokenize(text)
    # stem
    stems = stem_tokens(tokens, stemmer)
    return stems
######## 

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    stop_words = 'english',
    max_features = 550
)

In [20]:
corpus_data_features = vectorizer.fit_transform(train_data_df.Text.tolist() + test_data_df.Text.tolist())


In [11]:
corpus_data_features_nd = corpus_data_features.toarray()
corpus_data_features_nd.shape

(797952L, 85L)

In [12]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print vocab

[u'amp', u'awesom', u'best', u'better', u'bit', u'com', u'come', u'cool', u'd', u'day', u'did', u'don', u'feel', u'final', u'follow', u'friend', u'fun', u'gon', u'good', u'got', u'great', u'guy', u'ha', u'haha', u'happi', u'hey', u'hi', u'home', u'hope', u'http', u'im', u'just', u'know', u'let', u'like', u'll', u'lol', u'look', u'love', u'lt', u'm', u'make', u'morn', u'movi', u'na', u'need', u'new', u'nice', u'night', u'oh', u'onli', u'peopl', u'quot', u'realli', u'right', u's', u'say', u'sleep', u't', u'thank', u'thi', u'thing', u'think', u'time', u'today', u'tomorrow', u'tonight', u'tri', u'tweet', u'twitpic', u'twitter', u'u', u've', u'veri', u'wa', u'wait', u'want', u'watch', u'way', u'week', u'work', u'x', u'yay', u'ye', u'yeah']


In [13]:
# Sum up the counts of each vocabulary word
dist = np.sum(corpus_data_features_nd, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

26605 amp
15101 awesom
11334 best
12563 better
14883 bit
35366 com
19276 come
10750 cool
13412 d
56396 day
12426 did
17543 don
14514 feel
10826 final
20285 follow
13953 friend
18516 fun
10845 gon
62009 good
32174 got
25552 great
11347 guy
16866 ha
21729 haha
20691 happi
13975 hey
14558 hi
15959 home
24594 hope
46977 http
21178 im
63140 just
27451 know
10926 let
40998 like
24791 ll
36384 lol
22036 look
61564 love
11931 lt
62016 m
21546 make
20216 morn
10732 movi
16490 na
16928 need
26943 new
17437 nice
24590 night
17544 oh
10623 onli
10642 peopl
49155 quot
18708 realli
13036 right
99702 s
13905 say
11476 sleep
59902 t
51139 thank
41994 thi
13064 thing
24366 think
34610 time
30336 today
14973 tomorrow
12734 tonight
11818 tri
14005 tweet
14168 twitpic
21571 twitter
33216 u
12172 ve
14014 veri
45889 wa
18641 wait
17274 want
25184 watch
13094 way
12033 week
28796 work
11096 x
10628 yay
13021 ye
12808 yeah


In [14]:
###   A bag-of-words linear classifier
#In order to perform logistic regression in Python we use sklearn.linear_model.LogisticRegression. 
#But first let's split our training data in order to get an evaluation set. 
#We will use sklearn.cross_validation.train_test_split.

from sklearn.cross_validation import train_test_split

# remember that corpus_data_features_nd contains all of our original train and test data, so we need to exclude
# the unlabeled test entries
X_train, X_test, y_train, y_test  = train_test_split(
    corpus_data_features_nd[0:len(train_data_df)], 
    train_data_df.Sentiment,
    train_size=0.85, 
    random_state=1234)

C:\Users\hp\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [15]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression()
log_model = log_model.fit(X=X_train, y=y_train)

In [16]:
 y_pred = log_model.predict(X_test)


In [17]:
#the function sklearn.metrics.classification_report calculates several types of (predictive) scores on a classification model
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.67      0.81      0.73       763
          1       0.75      0.58      0.65       737

avg / total       0.71      0.70      0.69      1500



In [21]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.69799999999999995

In [18]:
#Finally, we can re-train our model with all the training data and use it for sentiment 
#classification with the original (unlabeled) test set.

# train classifier
log_model = LogisticRegression()
log_model = log_model.fit(X=corpus_data_features_nd[0:len(train_data_df)], y=train_data_df.Sentiment)

# get predictions
test_pred = log_model.predict(corpus_data_features_nd[len(train_data_df):])

# sample some of them
import random
spl = random.sample(xrange(len(test_pred)), 15)

# print text and labels
for text, sentiment in zip(test_data_df.Text[spl], test_pred[spl]):
    print sentiment, text

0 exam shutdown 
1 @elfia thank you! hope to see u soon comadre! 
0 @saborKT If you do go with that haircut post pictures. We all deserve to see how hot you'll look 
1 @bradwilson oh...ice road truckers is awesome btw 
0 mariah got 3rd in the 100 and 5th in the long jump!! woooo muh-rye-uhh!! 
1 [Utterli] http://bit.ly/cREjI  Good Morning: Welcome to a &quot;Thankful Thursday&quot;. I hope you got the memo 
0 @SingleOrigin totally my pleasure 
1 WOW!! Jaguar Skills megamix up for download, 30 yrs of hip-hop, 538 tracks: http://twurl.nl/2ia1rt cheers @tag 
0 @wahliaodotcom Actually don't sound like a bad idea to me. Why waste the shots. 
0 Going to bed now. Goodnight people 
0 @woodaledesigns No ideea lol...go on www.blacknight.com and take it if you want 
0 @fitprosarah is there any other way to eat a Hostess!!! 
0 Okej, Iï¿½m bored... So I write a little  But Iï¿½m really tired and the back of my neck hurts like *****! So I think I should go to bed soon.
0 @TheIronPenguin Unbelievable

In [ ]:

"""

conclusion

So judge by yourself. Is our classifier doing a good job at all? Considering how small our training data is, 
first we are getting a decent accuracy in the eval split, and second, when getting a sample of predictions for the test set,
most of the tags make sense. It would be great to find a larger training dataset with labels.
Doing so we will be able to train a better model and also to have more data
to split into train/eval sets and check our model accuracy.

In any case, we have used very simple methods here.
And mostly using default parameters. There is a lot of space for improvement in both areas.
We can fine tune each library parameters and we can also 
try more sophisticated parameters (e.g. Random Forests are very powerful). 
Additionally, we can use a different sparsity coefficient when selecting important words.
Our models just considered 85 words. We might try increasing 
this number to take into account more words (or less?) and see how the accuracy changes.

Finally, although the purpose of these tutorials is not to 
find a winer between R and Python but to show that there are just problems to solve 
and methods that can use with both platforms, we find here some differences.
In the case of R for example we have a nice summary function that we can use with the result of training a linear classifier.
This summary shows us very important information regarding how significative each feature is (i.e. each word in our case). 
Also the text analysis process seems more straightforward in R. But Python as usual is 
more structured and granular, and is easier to adapt to our particular needs by plugging and pipelining 
different parts of the process.

Where to go from here? The next step should be to put one of our models into a data product people can use. 
For example, we could use the R model we just built to create a web application where we can
submit text and get a sentiment classification estimate. For that we could use 
the Shiny platform, a great way to quickly create and share data products as web applications.
"""
